<a href="https://colab.research.google.com/github/anikch/Anik424/blob/main/Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 1.5 MB/s 
     |████████████████████████████████| 6.8 MB 35.4 MB/s 
     |████████████████████████████████| 981 kB 45.4 MB/s 
     |████████████████████████████████| 263 kB 57.9 MB/s 
     |████████████████████████████████| 1.9 MB 33.8 MB/s 
     |████████████████████████████████| 1.2 MB 39.7 MB/s 
     |████████████████████████████████| 468 kB 49.2 MB/s 
     |████████████████████████████████| 3.3 MB 54.0 MB/s 
     |████████████████████████████████| 895 kB 34.4 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=16c72e530d5616d7733319bcbb95163383df953e375c4787058e8f0d36f56522
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=2bdd2879663b599644a08bc62322e9c02a680bbb585c1c764a63a9cc5b14f5dd
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff

In [3]:
import pandas as  pd
import numpy as np
import xlrd
import sqlite3
import ktrain
import tensorflow as tf
from sklearn.model_selection import train_test_split
from ktrain import text

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
final_df= pd.read_csv('/content/gdrive/MyDrive/colab_data/final.csv')

In [6]:
final_df.head()

,text,sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [7]:
tf.version.VERSION

'2.6.0'

In [8]:
final_df.dropna(inplace= True)
final_df.sentiment.value_counts()

positive    157179
negative    120999
neutral     106104
Name: sentiment, dtype: int64

In [9]:
X_train, X_test, y_train, y_test= train_test_split(final_df['text'], final_df['sentiment'], train_size= .8, 
                                                   random_state= 42, stratify= final_df['sentiment'])

In [10]:
factors = ['negative', 'neutral', 'positive']

encoding= { 'negative': 0, 'neutral': 1, 'positive': 2}

In [11]:
y_train= [encoding[key] for key in y_train]
y_test= [encoding[key] for key in y_test]

In [12]:
X_train= np.array(X_train)
X_test= np.array(X_test)
y_train= np.array(y_train)
y_test= np.array(y_test)

In [13]:
model_arch='bert-base-uncased'
MAXLEN= 256
trans= text.Transformer(model_arch, maxlen=MAXLEN, class_names= factors)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [14]:
train_data= trans.preprocess_train(X_train, y_train)

preprocessing train...
language: en
train sequence lengths:
	mean : 18
	95percentile : 38
	99percentile : 65


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Is Multi-Label? False


In [15]:
test_data= trans.preprocess_test(X_test,y_test)

preprocessing test...
language: en
test sequence lengths:
	mean : 18
	95percentile : 38
	99percentile : 68


In [16]:
model= trans.get_classifier()

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

In [17]:
learner= ktrain.get_learner(model, train_data= train_data, val_data= test_data, batch_size= 10)

In [18]:
#learner.lr_find(show_plot=True, max_epochs= 10)

In [19]:
learner.fit_onecycle(3e-5, 2)



begin training using onecycle policy with max lr of 3e-05...
Epoch 1/2
30743/30743 [==============================] - 11419s 368ms/step - loss: 0.3499 - accuracy: 0.8598 - val_loss: 0.2158 - val_accuracy: 0.9191
Epoch 2/2
30743/30743 [==============================] - 11380s 368ms/step - loss: 0.1461 - accuracy: 0.9465 - val_loss: 0.1186 - val_accuracy: 0.9571


In [21]:
learner.model.save_pretrained('/content/gdrive/MyDrive/colab_data/sent_model1')

In [25]:
predictor= ktrain.get_predictor(learner.model, preproc= trans)

In [26]:
predictor.save('/content/gdrive/MyDrive/colab_data/newmodel/bert_sent')

In [27]:
loaded_model= ktrain.load_predictor('/content/gdrive/MyDrive/colab_data/newmodel/bert_sent/')

In [67]:
def predict_sentiment(inp):
  labels_li= ['negative', 'neutral', 'positive']
  pred_val= loaded_model.predict(inp, return_proba= True)
  sent=  labels_li[np.argmax(pred_val)]

  return {'confidence': 
          {
            'negative': pred_val[0],
            'neutral': pred_val[1],
            'positive': pred_val[2]}, 
          'sentiment': sent}


In [77]:
inp= "Seriously? are you joking?"

In [78]:
predict_sentiment(inp)

{'confidence': {'negative': 0.55782247,
  'neutral': 0.3749075,
  'positive': 0.06727006},
 'sentiment': 'negative'}